# **ONSAGER TESTING**

## **Installs**

### Firedrake

In [1]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2026-02-04 17:49:30--  https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4775 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2026-02-04 17:49:30 (44.3 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4775/4775]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#########################################################

### Gmsh

In [2]:
try:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh"
    !bash "/tmp/gmsh-install.sh"
    import gmsh  # noqa: F401
except:
    import gmsh  # noqa: F401

--2026-02-04 17:49:35--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3497 (3.4K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   3.42K  --.-KB/s    in 0s      

2026-02-04 17:49:35 (49.3 MB/s) - ‘/tmp/gmsh-install.sh’ saved [3497/3497]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ set +x
























################################################################################
#     This installation

### Other

In [3]:
import numpy as np
import matplotlib.pyplot as plt

## **Code**

### Mesh

In [4]:
def rect_with_hole_mesh(r=0.5, left=1.0, right=4.0, updown=1.0, h=0.2):
    '''
    r = hole radius
    left = space to left of hole centre
    right = space to right of hole centre
    updown = space above / below hole centre
    h = (target) mesh size
    '''
    gmsh.initialize()
    gmsh.model.add("rect_with_hole")

    # Derived rectangle bounds
    xmin = -left
    xmax = right
    ymin = -updown
    ymax = updown


    # -------------------
    # Rectangle
    # -------------------
    p1 = gmsh.model.geo.addPoint(xmin, ymin, 0, h)
    p2 = gmsh.model.geo.addPoint(xmax, ymin, 0, h)
    p3 = gmsh.model.geo.addPoint(xmax, ymax, 0, h)
    p4 = gmsh.model.geo.addPoint(xmin, ymax, 0, h)

    l_bottom = gmsh.model.geo.addLine(p1, p2)
    l_right  = gmsh.model.geo.addLine(p2, p3)
    l_top    = gmsh.model.geo.addLine(p3, p4)
    l_left   = gmsh.model.geo.addLine(p4, p1)

    rect_loop = gmsh.model.geo.addCurveLoop(
        [l_bottom, l_right, l_top, l_left]
    )


    # -------------------
    # Circular hole (centred at origin)
    # -------------------
    c  = gmsh.model.geo.addPoint(0, 0, 0, h)
    cp = gmsh.model.geo.addPoint( r, 0, 0, h)
    cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
    ct = gmsh.model.geo.addPoint(0,  r, 0, h)
    cb = gmsh.model.geo.addPoint(0, -r, 0, h)

    a1 = gmsh.model.geo.addCircleArc(cp, c, ct)
    a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
    a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
    a4 = gmsh.model.geo.addCircleArc(cb, c, cp)

    hole_loop = gmsh.model.geo.addCurveLoop([a1, a2, a3, a4])


    # -------------------
    # Surface with hole
    # -------------------
    surface = gmsh.model.geo.addPlaneSurface([rect_loop, hole_loop])

    gmsh.model.geo.synchronize()


    # -------------------
    # Physical groups (THIS is the important bit)
    # -------------------
    gmsh.model.addPhysicalGroup(1, [l_left], tag=1)
    gmsh.model.setPhysicalName(1, 1, "inlet")

    gmsh.model.addPhysicalGroup(1, [l_right], tag=2)
    gmsh.model.setPhysicalName(1, 2, "outlet")

    gmsh.model.addPhysicalGroup(1, [l_top, l_bottom], tag=3)
    gmsh.model.setPhysicalName(1, 3, "wall")

    gmsh.model.addPhysicalGroup(1, [a1, a2, a3, a4], tag=4)
    gmsh.model.setPhysicalName(1, 4, "hole")

    gmsh.model.addPhysicalGroup(2, [surface], tag=5)


    # -------------------
    # Mesh
    # -------------------
    gmsh.model.mesh.generate(2)
    gmsh.write("rect_with_hole.msh")
    gmsh.finalize()

    return Mesh("rect_with_hole.msh")

### Navier–Stokes

In [21]:
def navier_stokes(h=0.2, degree=2, sigma=10.0, Re=1.0, dt=2**-5, T=2**-3, download=False):
    # ---------
    #   Setup
    # ---------
    # Mesh
    mesh = rect_with_hole_mesh(h=h)
    n = FacetNormal(mesh)

    # Normalise parameters
    h_c = Constant(h)
    degree_c = Constant(degree)
    sigma_c = Constant(sigma)
    Re_c = Constant(Re)
    dt_c = Constant(dt)

    # Spaces
    V = FunctionSpace(mesh, "BDM", degree)
    Q = FunctionSpace(mesh, "DG", degree-1)
    W = V * Q

    # Unknowns
    w = Function(W)
    u, p = split(w)
    u_sol, p_sol = w.subfunctions
    u_sol.rename("velocity")
    p_sol.rename("pressure")

    # Old velocity
    u_old = Function(V)

    # Tests
    v, q = TestFunctions(W)

    # Solver parameters
    sp = {
        "snes_monitor" : None,
        "snes_converged_reason" : None,
    }


    # -----------------------
    #   Boundary conditions
    # -----------------------
    bcs = [
        DirichletBC(W.sub(0), Constant((1.0, 0.0)), 1),
        DirichletBC(W.sub(0), Constant((0.0, 0.0)), 3),
        DirichletBC(W.sub(0), Constant((0.0, 0.0)), 4),
    ]


    # ------------------------
    #   Initial Stokes solve
    # ------------------------
    # Operators
    dev = lambda u : sym(grad(u))
    jump = lambda u : outer(u, n)('+') + outer(u, n)('-')

    # Viscous term
    viscous_bulk = 2 / Re_c * inner(dev(u), dev(v)) * dx
    viscous_int = (
      - 2 / Re_c * inner(avg(dev(u)), sym(jump(v)))
      - 2 / Re_c * inner(sym(jump(u)), avg(dev(v)))
      + sigma_c * inner(sym(jump(u)), sym(jump(v)))
    ) * dS

    # Residual
    F = (
        viscous_bulk + viscous_int
      - inner(p, div(v)) * dx
      - inner(q, div(u)) * dx
    )

    # PVD setup
    if download: outfile = VTKFile("velocity.pvd")

    # Solve
    print(GREEN % f"Setting up ICs for t = 0:")
    solve(F == 0, w, bcs=bcs, solver_parameters=sp)
    if download: outfile.write(u_sol, p_sol)
    u_old.assign(u_sol)


    # ----------------------------
    #   Full Navier-Stokes solve
    # ----------------------------
    # Advective term
    advective_bulk = inner(dot(grad(u), u), v) * dx
    advective_int  = - inner(dot(jump(u), avg(u)), avg(v)) * dS

    # Residual
    F += (
        1 / dt_c * inner((u - u_old), v) * dx
      + advective_bulk + advective_int
    )

    # Time loop
    t = 0.0
    step = 0
    while t < T:
        t += float(dt)
        step += 1
        print(GREEN % f"Solving for t = {t:.4e}:")
        solve(F == 0, w, bcs=bcs, solver_parameters=sp)
        if download: outfile.write(u_sol, p_sol)
        u_old.assign(u_sol)


    # ------------
    #   Download
    # ------------
    if download:
        import zipfile
        import glob
        import os
        from google.colab import files

        # Zip the .pvd and all .vtu files
        zip_name = "velocity_data.zip"
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            if os.path.exists("velocity.pvd"):
                zipf.write("velocity.pvd")

            # The .vtu files are stored in a folder called 'velocity'
            if os.path.isdir("velocity"):
                for root, _, files_list in os.walk("velocity"):
                    for file in files_list:
                        if file.endswith(".vtu"):
                            file_path = os.path.join(root, file)
                            zipf.write(file_path, arcname=file_path)

        files.download(zip_name)
        print(BLUE % f"Download complete!")

In [20]:
navier_stokes(Re=10.0, sigma=2.0, dt=0.001, T=0.01, degree=2, download=True)

Step 1 | t = 1.0000e-03
  0 SNES Function norm 2.119902391853e+02
  1 SNES Function norm 9.460810633098e+00
  2 SNES Function norm 2.609231684662e-04
  3 SNES Function norm 5.570574206113e-11
  Nonlinear petsctools_101_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 3
Step 2 | t = 2.0000e-03
  0 SNES Function norm 5.570574206113e-11
  1 SNES Function norm 5.373898635008e-11
  Nonlinear petsctools_102_ solve converged due to CONVERGED_SNORM_RELATIVE iterations 1
Step 3 | t = 3.0000e-03
  0 SNES Function norm 5.373898635008e-11
  1 SNES Function norm 5.078333870487e-11
  Nonlinear petsctools_103_ solve converged due to CONVERGED_SNORM_RELATIVE iterations 1
Step 4 | t = 4.0000e-03
  0 SNES Function norm 5.078333870487e-11
  1 SNES Function norm 5.231600876299e-11
  Nonlinear petsctools_104_ solve converged due to CONVERGED_SNORM_RELATIVE iterations 1
Step 5 | t = 5.0000e-03
  0 SNES Function norm 5.231600876299e-11
  1 SNES Function norm 4.822016516060e-11
  Nonlinear petsctoo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download complete!
